In [1]:
import os
import sys

# Get the current working directory and add the parent directory to the Python path
current_working_directory = os.getcwd()
print(os.path.join(current_working_directory, ".."))
sys.path.append(os.path.join(current_working_directory, ".."))

/Users/L024258/lilly_work/github-copilot/exploration/applications/..


In [2]:
from decouple import AutoConfig
config = AutoConfig(search_path='./../.env')

os.environ["AZURE_OPENAI_API_KEY"] = config('AZURE_OPENAI_API_KEY')
os.environ["AZURE_OPENAI_ENDPOINT"] = config('AZURE_ENDPOINT')

In [3]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

urls = ('https://2024.emnlp.org/program/accepted_main_conference/', 
        )
tag = ('strong')
tag_classes = ()

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=urls,
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(tag,
            class_=tag_classes
        )
    ),
)

docs = loader.load_and_split()
doc = docs[0]
print(len(docs))

USER_AGENT environment variable not set, consider setting it to identify your requests.


28


In [4]:
print(doc.page_content)

UniGen: Universal Domain Generalization for Sentiment Classification via Zero-shot Dataset GenerationMulti-News+: Cost-efficient Dataset Cleansing via LLM-based Data AnnotationFIZZ: Factual Inconsistency Detection by Zoom-in Summary and Zoom-out DocumentPrompts have evil twinsTable Question Answering for Low-resourced Indic LanguagesImageInWords: Unlocking Hyper-Detailed Image DescriptionsLLM-Based Agent Society Investigation: Collaboration and Confrontation in Avalon GameplayWhen LLMs Meets Acoustic Landmarks: An Efficient Approach to Integrate Speech into Large Language Models for Depression DetectionSpeaking in Wavelet Domain: A Simple and Efficient Approach to Speed up Speech Diffusion ModelHateful Word in Context ClassificationEyes Don’t Lie: Subjective Hate Annotation and Detection with GazeNumeroLogic: Number Encoding for Enhanced LLMs’ Numerical ReasoningThinking Fair and Slow: On the Efficacy of Structured Prompts for Debiasing Language ModelsA Usage-centric Take on Intent Und

In [5]:
from pydantic import BaseModel, Field
class Overview(BaseModel):
    """Overview of a section of text."""
    summary: str = Field(description="Provide a detailed summary of the content.")
    highlights: str = Field(description="Provide the highlights of the content.")
    keywords: str = Field(description="Provide keywords related to the content.")

In [6]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("human", "{input}")
])

trend_prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("human", """Analyse and Extract the key themes/topics from the provided accepted papers in the confernce. 
     Also provide the papers and keywords related to the topic.
     {input}""")
])

In [7]:
from langchain_openai import AzureChatOpenAI
model = AzureChatOpenAI(
                openai_api_version=config('AZURE_CHAT_OPENAI_API_VERSION'),
                azure_deployment=config('AZURE_GPT4o_mini_CHAT_OPENAI_DEPLOYMENT'),
                temperature=0,
                max_tokens=4096
            )

In [8]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

overview_tagging_function = [
    convert_pydantic_to_openai_function(Overview)
]
tagging_model = model.bind(
    functions=overview_tagging_function,
    function_call={"name":"Overview"}
)
tagging_chain = prompt | tagging_model | JsonOutputFunctionsParser()
trend_chain = trend_prompt | model

/var/folders/q0/sfx4jy8s5bx23xxtplmtrj5c0000gp/T/ipykernel_50309/3622901728.py:5: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  convert_pydantic_to_openai_function(Overview)


In [9]:
result = tagging_chain.invoke({"input": docs})
for key, value in result.items():
    print(f"{key}")
    print(f"{value}")

summary
The document contains a comprehensive list of accepted papers for the EMNLP 2024 conference, covering a wide range of topics related to large language models (LLMs), multimodal models, and various applications in natural language processing (NLP). Each entry includes the title of the paper, which reflects current research trends and challenges in the field, such as bias detection, reasoning capabilities, data augmentation, and model evaluation.
highlights
1. Papers focus on advancements in LLMs and their applications in various domains, including sentiment analysis, factuality detection, and multimodal understanding.
2. Topics include bias mitigation, instruction tuning, and the integration of external knowledge into LLMs.
3. Research addresses challenges like hallucinations in LLMs, the effectiveness of prompts, and the evaluation of model performance across different tasks.
keywords
Large Language Models, Multimodal Models, Natural Language Processing, Bias Detection, Instruc

In [10]:
result = trend_chain.invoke({"input": docs})
result

AIMessage(content='Here are the key themes/topics extracted from the accepted papers at the conference, along with related papers and keywords:\n\n### 1. **Large Language Models (LLMs) and Their Applications**\n   - **Papers:**\n     - "UniGen: Universal Domain Generalization for Sentiment Classification via Zero-shot Dataset Generation"\n     - "LLM-Based Agent Society Investigation: Collaboration and Confrontation in Avalon Gameplay"\n     - "Fine-Tuning Large Language Models to Translate: Will a Touch of Noisy Data in Misaligned Languages Suffice?"\n   - **Keywords:** LLMs, fine-tuning, zero-shot learning, sentiment classification, translation.\n\n### 2. **Bias and Fairness in AI**\n   - **Papers:**\n     - "Thinking Fair and Slow: On the Efficacy of Structured Prompts for Debiasing Language Models"\n     - "On the Influence of Gender and Race in Romantic Relationship Prediction from Large Language Models"\n     - "Mitigating the Alignment Tax of RLHF"\n   - **Keywords:** bias, fair

In [11]:
print(result.content)

Here are the key themes/topics extracted from the accepted papers at the conference, along with related papers and keywords:

### 1. **Large Language Models (LLMs) and Their Applications**
   - **Papers:**
     - "UniGen: Universal Domain Generalization for Sentiment Classification via Zero-shot Dataset Generation"
     - "LLM-Based Agent Society Investigation: Collaboration and Confrontation in Avalon Gameplay"
     - "Fine-Tuning Large Language Models to Translate: Will a Touch of Noisy Data in Misaligned Languages Suffice?"
   - **Keywords:** LLMs, fine-tuning, zero-shot learning, sentiment classification, translation.

### 2. **Bias and Fairness in AI**
   - **Papers:**
     - "Thinking Fair and Slow: On the Efficacy of Structured Prompts for Debiasing Language Models"
     - "On the Influence of Gender and Race in Romantic Relationship Prediction from Large Language Models"
     - "Mitigating the Alignment Tax of RLHF"
   - **Keywords:** bias, fairness, debiasing, gender, race, al